############## HAZY IPA SCRAPE (also know as new-england or juicy IPA) ################
unfortunately there isn't a hazy ipa/ NEIPA/ juicy IPA style category on brewersfriend so 

In [88]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys 

driver = webdriver.Chrome()  
driver.get('https://www.brewersfriend.com/search/')

#close popup
xpath = "//*[@class='fancybox-item fancybox-close']"
WebDriverWait(driver, 35).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
#time.sleep(35) #Should use webDriverWait here instead - takes awhile for popup to appear
#driver.find_element_by_xpath("//*[@class='fancybox-item fancybox-close']").click()

#find all search bars available
#searches[0] = method,... searches[1,2,3,4,5,6] = [units,style,fermentables,hops,yeasts,other]
time.sleep(1)
searches = driver.find_elements_by_xpath("//*[@class='search']")

#input brew method
searches[0].clear()                #clear any populated text (there isn't any in this case but its good practice)
searches[0].send_keys('All Grain') #input all grain into searchbar
searches[0].send_keys(Keys.ENTER)  #hit enter

#input brew style
searches[2].clear()
searches[2].send_keys('Specialty IPA: New England IPA')
searches[2].send_keys(Keys.ENTER)

#get total pages of recipes
time.sleep(2) #use webDriverWait eventually
totalPages = int(driver.find_element_by_xpath("//*[contains(text(), 'Last')]").get_attribute('data-page-number'))

recipeLinks = [] # initialize list to store recipeLinks

#iterate through all of the pages and get all recipe links on each page
currentPage = 0 #initialize current page before entering while loop
#emulate do-while loop
while True:
    currentPage = currentPage + 1
    time.sleep(2) #give it time after clicking next page to avoid stale element error
    recipes = driver.find_elements_by_xpath("//*[@class='recipetitle']")

    #get links for each recipe
    for i in range(0,(len(recipes))):
        recipeLinks.append(recipes[i].get_attribute("href"))
     
    #check to see if on last page and break from while if so, 
    #otherwise, go to next page
    if (currentPage == totalPages):
        break
    
    #go to next page
    time.sleep(random.random()*5+1)
    #click the last available next (avoid clicking links with recipe names that have 'next' in them)
    nexts = driver.find_elements_by_xpath("//*[contains(text(), 'Next')]") 
    nexts[len(nexts)-1].click() 


In [91]:
#print number of recipes and save recipe links to csv
print("Number of recipes: ", len(recipeLinks))
df = pd.DataFrame(recipeLinks, columns = ['NEIPA links'])
df.head()
#df.to_csv('NEIPAlinks.csv', encoding = 'utf-8', index = False)


Number of recipes:  2854


,NEIPA links
0,https://www.brewersfriend.com/homebrew/recipe/...
1,https://www.brewersfriend.com/homebrew/recipe/...
2,https://www.brewersfriend.com/homebrew/recipe/...
3,https://www.brewersfriend.com/homebrew/recipe/...
4,https://www.brewersfriend.com/homebrew/recipe/...


In [ ]:
import time
import sys
import pandas as pd
import random
import csv
import urllib.request
import requests
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from itertools import cycle
import traceback
import matplotlib.pyplot as plt
import numpy as np
import re

In [168]:
#now that we have the links scraped, we can start 
#scraping ingredients and other pertinent info

start_time = time.time()

#initialize all parameters
#awardWin = [] #binary yes or no if its an award winner
recipeURL = []
name = []
author = [] #creator of recipe

#beer stats
method = [] #brew method (all grain, extract, etc)
style = [] #brew style (NEIPA, cream ale, etc)
boilTime = []
batchSize = [] #fermentor volume
preBoilSize = []
postBoilSize = []
preBoilGravity = [] #recipe based estimate
efficiency = []
source = []
noChill = [] #letting it cool by itself with no external help
rating = []
numRating = [] #number of people who rated beer
hopUtil = [] #hop utilization
calories = []
carbs = []
views = [] #number of views recipe has
brews = [] #number of other users that have brewed the beer
links = [] #url might be present, for example as proof for award winners
dateCreated = [] #date recipe was created

og = [] #original gravity
fg = [] #final gravity
abv = [] #alcohol by volume
ibu = [] #international bitterness units
srm = [] #standard reference method (beer color)
mashPh = [] #ph of mash

fermNames = []
fermAmounts = []
fermPPGs    = [] #points per pound per gallon
fermLs      = [] #°L (degrees Lovibond)
fermBillPs  = [] #bill percentage

hopAmounts = []
hopNames = []
hopForms = [] #i.e., leaf/whole or pellet
hopAAs = [] #alpha acid content
hopUses = [] #when added i.e., during boil, dry hop, etc
hopTimes = [] #when to add hops
hopIBUs = [] #international bitterness units
hopBillPs = [] #bill percentage

yeastNames = []
yeastAmounts = []
yeastAttenuations = [] #yeast attenuation (avg)
yeastOptTemps = [] #optimum temp
yeastFlocs = [] #flocculation
yeastStarters = [] #binary yes or no
yeastFermTemps = [] #fermentation temp
yeastPRs = [] #pitch rate

#target water profile
ca = [] #Ca(+2)
mg = [] #Mg(+2)
na = [] #Na(+)
cl = [] #Cl(-)
so4 = [] #SO4(-2)
hco3 = [] #HCO3(-)

priming = [] #priming info (how to carbonate)

otherNames = []
otherAmounts = []
otherTypes = []
otherUses = []
otherTimes = []

notes = [] #any additional notes author provided

data = pd.read_csv('NEIPAlinks 1-26-20.csv', encoding = "utf-8")
count = 0
#loop through each recipe
for link in data['NEIPA links'][2000:len(data['NEIPA links'])]:#len(data['NEIPA links'])
    
    #loop through each recipe
    url = link
    #count = count+1
    count = data[data['NEIPA links'] == url].index[0] #index of url = count+1
    
    recipeURL.append(url)
    hdr = {'User-Agent': 'Mozilla/5.0'} # used to avoid 403 error
    req = Request(url, headers=hdr)
    try:
        page = urlopen(req)
    except HTTPError as e:
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
        print(link)
        sys.exit(1)
    except URLError as e:
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
        print(link)
        sys.exit(1)
    else:
        # everything is fine
        soup = BeautifulSoup(page, "lxml")
        
        #get name of recipe
        name_container = soup.find('h3', itemprop='name')
        if name_container is None:
            name.append('NA')
        else:
            name.append(name_container.text.strip())

        #get author of recipe
        author_container = soup.find('span', itemprop = "author")
        if author_container is None:
            author.append('NA')
        else:
            author.append(author_container.text.strip())

        #get date created
        date_container = soup.find('div', class_="extra") 
        if date_container is None:
            dateCreated.append('NA')
        else:
            dateCreated.append(date_container.text.strip().replace('Created', ''))

        #get beer stats
        beerStats_container = soup.find('div', class_ = "description")
        if beerStats_container is None:
            method.append('NA')
            style.append('NA')
            boilTime.append('NA')
            batchSize.append('NA')
            preBoilSize.append('NA')
            postBoilSize.append('NA')
            preBoilGravity.append('NA')
            efficiency.append('NA')
            source.append('NA')
            noChill.append('NA')
            rating.append('NA')
            numRating.append('NA')
            hopUtil.append('NA')
            calories.append('NA')
            carbs.append('NA')
            links.append('NA')
        else:
            for i in range(0,len(beerStats_container.find_all('span', class_='viewStats'))):
                if beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Method:':
                    method.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Style:':
                    style.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Boil Time:':
                    boilTime.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Batch Size:':
                    batchSize.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Pre Boil Size:':
                    preBoilSize.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Post Boil Size:':
                    postBoilSize.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Pre Boil Gravity:':
                    preBoilGravity.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Efficiency:':
                    efficiency.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Source:':
                    source.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Rating:':
                    rating.append(beerStats_container.find_all('span', class_='viewStats')[i].find('span',itemprop = "ratingValue").text.strip())
                    numRating.append(beerStats_container.find_all('span', class_='viewStats')[i].find('span',itemprop = "reviewCount").text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'No Chill:':
                    noChill.append(' '.join(beerStats_container.find_all('span', class_='viewStats')[i].text.split()).split('No Chill: ',1)[1])
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Calories:':
                    calories.append(beerStats_container.find_all('span', class_='viewStats')[i].text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Hop Utilization:':
                    hopUtil.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Carbs:':
                    carbs.append(beerStats_container.find_all('span', class_='viewStats')[i].text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'URL:':
                    links.append(beerStats_container.find_all('span', class_='viewStats')[i].find('a').text.strip())
                else:
                    print("Theres another general stat you're not aware of: ", url)

            #assign NA to any missing stats by making a list of whats present and checking if any are missing
            statNames = [] 
            for i in range(0, len(beerStats_container.find_all('span', class_='viewStats'))):
                statNames.append(beerStats_container.find_all('span', class_='viewStats')[i].find('span',class_="firstLabel").text)

            if 'Method:' not in statNames:
                method.append('NA')
            if 'Style:' not in statNames:
                style.append('NA')
            if 'Boil Time:' not in statNames:
                boilTime.append('NA')
            if 'Batch Size:' not in statNames:
                batchSize.append('NA')
            if 'Pre Boil Size:' not in statNames:
                preBoilSize.append('NA')
            if 'Post Boil Size:' not in statNames:
                postBoilSize.append('NA')
            if 'Pre Boil Gravity:' not in statNames:
                preBoilGravity.append('NA')
            if 'Efficiency:' not in statNames:
                efficiency.append('NA')
            if 'Source:' not in statNames:
                source.append('NA')
            if 'Rating:' not in statNames:
                rating.append('NA')
                numRating.append('NA')
            if 'Hop Utilization:' not in statNames:
                hopUtil.append('NA')
            if 'No Chill:' not in statNames:
                noChill.append('NA')
            if 'Calories:' not in statNames:
                calories.append('NA')
            if 'Carbs:' not in statNames:
                carbs.append('NA')
            if 'URL:' not in statNames:
                links.append("NA")

        #get remaining stats in bar below main stats
        beerStatsOther_container = soup.find('div', class_='appbounds minstats')
        if beerStatsOther_container is None:
            og.append('NA')
            fg.append('NA')
            abv.append('NA')
            ibu.append('NA')
            srm.append('NA')
            mashPh.append('NA')
        else: 
            #need to convert degrees plato to specific gravity: SG = 1+ (dP / (258.6 – ( (dP/258.2) *227.1) ) )
            if '°P' in beerStatsOther_container.find('div', class_='value ogBatch').text.strip():
                dP_og = beerStatsOther_container.find('div', class_='value ogBatch').text.strip().split(' °P')[0]
                dP_fg = beerStatsOther_container.find('div', class_='value fgBatch').text.strip().split(' °P')[0]
                og.append(round(1 + (dP_og/(258.6 – ((dP_og/258.2)*227.1))),3)
                fg.append(round(1 + (dP_fg/(258.6 – ((dP_fg/258.2)*227.1))),3)
            else:
                og.append(beerStatsOther_container.find('div', class_='value ogBatch').text.strip())
                fg.append(beerStatsOther_container.find('div', class_='value fgBatch').text.strip())
                
            abv.append(beerStatsOther_container.find('div', class_='value abvMin').text.strip().remove('%'))
            ibu.append(beerStatsOther_container.find('div', class_='value ibuMin').text.strip())
            srm.append(beerStatsOther_container.find('div', class_='value srmMin').text.strip())
            if beerStatsOther_container.find('div', class_='value phMin').text.strip()=='n/a':
                mashPh.append('NA')
            else:
                mashPh.append(beerStatsOther_container.find('div', class_='value phMin').text.strip())

        # get fermentable ingredient list
        fermAmount = []
        fermName   = [] 
        fermPPG    = [] #points per pound per gallon
        fermL      = [] #°L (degrees Lovibond)
        fermBillP  = [] #bill percentage

        ferm_container = soup.find('div', class_ = "brewpart", id = 'fermentables') #location of fermentables
        if ferm_container is None: #if fermentables aren't available, record 'NA'
            fermAmount.append('NA')
            fermName.append('NA')
            fermPPG.append('NA')
            fermL.append('NA')
            fermBillP.append('NA')
        else:          #else access specific partsof table where data are recorded, # i = table row, j = table column 
            for i in range(1,len(ferm_container.table.find_all('tr'))-1): #-1 to avoid last row (total weight)  
                for j in range(0,len(ferm_container.table.find_all('tr')[i])):
                    if j == 0: #amounts are in 1st column
                        fermAmount.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 1: #names in 2nd column...etc
                        #some recipes have the grain names hyperlinked, if hyperlinked there will be two
                        #text fields: the hop name and amount of hops
                        #e.g., this allows it to scrape just 'American - Wheat' instead of 'American - Wheat1.4 lb Wheat'
                        if (ferm_container.table.find_all('tr')[i].find_all('td')[j].find('a') is None):
                            fermName.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].find('span').text.strip())
                        else:
                            fermName.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].find('a').text.strip())
                    elif j == 2: #ppg
                        fermPPG.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 3: #°L (degrees Lovibond)
                        fermL.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 4: #bill percentage
                        fermBillP.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].text.strip())   

        fermAmounts.append(fermAmount)
        fermNames.append(fermName)
        fermPPGs.append(fermPPG)
        fermLs.append(fermL)
        fermBillPs.append(fermBillP)

        # get hop lists
        hopAmount = []
        hopName = []
        hopForm = [] #i.e., leaf/whole or pellet
        hopAA = [] #alpha acid content
        hopUse = [] #when added i.e., during boil, dry hop, etc
        hopTime = []
        hopIBU = [] #international bitterness units
        hopBillP = [] #bill percentage

        hop_container = soup.find('div', class_ = "brewpart", id = 'hops')
        if hop_container is None: #if hop table doesnt exist, record 'NA'
            hopAmount.append('NA')
            hopName.append('NA')
            hopForm.append('NA')
            hopAA.append('NA')
            hopUse.append('NA')
            hopTime.append('NA')
            hopIBU.append('NA')
            hopBillP.append('NA')
        else:                     #else record hopnames and temp/times
            if hop_container.find('div', id='hopsSummary', style='display: none;') is not None: #if hops table isn't in summary view
                for i in range(1,len(hop_container.table.find_all('tr'))):  # i = table row, j = table column 
                    for j in range(0,len(hop_container.table.find_all('tr')[i])):
                        if j == 0: #amounts are in 1st column
                            hopAmount.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 1: #names in 2nd column...etc
                            #some recipes have the hop names hyperlinked, if hyperlinked there will be two
                            #text fields: the hop name and amount of hops
                            #e.g., this allows it to scrape just 'Citra' instead of 'Citra1 oz Citra Hops'
                            if hop_container.table.find_all('tr')[i].find_all('td')[j].find('a') is None:
                                hopName.append(hop_container.table.find_all('tr')[i].find_all('td')[j].find('span').text.strip())
                            else:
                                hopName.append(hop_container.table.find_all('tr')[i].find_all('td')[j].find('a').text.strip())
                        elif j == 2: #form
                            hopForm.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 3: #AA (degrees Lovibond)
                            hopAA.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 4: #Use
                            hopUse.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 5: #Time
                            hopTime.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 6: #IBU
                            hopIBU.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 7: #bill percentage
                            hopBillP.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())   
            else: #else hops is in summary view - smaller table
                for i in range(1,len(hop_container.table.find_all('tr'))-1):  #-1 since last row is total (dont need)
                    for j in range(0,len(hop_container.table.find_all('tr')[i])): # i = table row, j = table column 
                        if j == 0: #amounts are in 1st column
                            hopAmount.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 1: 
                            #some recipes have the hop names hyperlinked, if hyperlinked there will be two
                            #text fields: the hop name and amount of hops
                            #e.g., this allows it to scrape just 'Citra' instead of 'Citra1 oz Citra Hops'
                            if hop_container.table.find_all('tr')[i].find_all('td')[j].find('a') is None:
                                #names and form are in 2nd column when hops are in summary view
                                unparsed = hop_container.table.find_all('tr')[i].find_all('td')[j].find('span').text.strip()
                                name_ = unparsed.split('(')[0].strip() #name_ has underscore because name is list of recipe names
                                form = unparsed.split('(')[1].replace(')','').strip()
                                hopName.append(name_)
                                hopForm.append(form)
                            else:
                                #names and form are in 2nd column when hops are in summary view
                                unparsed = hop_container.table.find_all('tr')[i].find_all('td')[j].find('a').text.strip()
                                name_ = unparsed.split('(')[0].strip()
                                form = unparsed.split('(')[1].replace(')','').strip()
                                hopName.append(name_)
                                hopForm.append(form)
                        elif j == 2: #IBU
                            hopIBU.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 3: #bill percentage
                            hopBillP.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip()) 

                        #hop AA, use, and time aren't available in summarized table
                        hopAA.append('NA') #we can look up average AA content and add it later
                        hopUse.append('NA')
                        hopTime.append('NA')
                
                
        hopAmounts.append(hopAmount)
        hopNames.append(hopName)
        hopForms.append(hopForm)
        hopAAs.append(hopAA)
        hopUses.append(hopUse)
        hopTimes.append(hopTime)
        hopIBUs.append(hopIBU)
        hopBillPs.append(hopBillP)


        #get yeast info
        yeastName = []
        yeastAmount = []
        yeastAttenuation = [] #yeast attenuation (avg)
        yeastOptTemp = [] #optimum temp
        yeastFloc = [] #flocculation
        yeastStarter = [] #binary yes or no
        yeastFermTemp = [] #fermentation temp
        yeastPR = [] #pitch rate

        #get yeast  
        #need to fix so it can scrape multiple yeasts - cant find a recipe with 2 yeasts atm tho
        yeast_container = soup.find('div', class_ = "brewpart", id = 'yeasts')
        if yeast_container is None:
            yeastName.append('NA')
            yeastAmount.append('NA')
            yeastAttenuation.append('NA')
            yeastOptTemp.append('NA')
            yeastFloc.append('NA')
            yeastStarter.append('NA')
            yeastFermTemp.append('NA')
            yeastPR.append('NA')
        else:
            for i in range(0,len(yeast_container.find_all('tbody'))): #most recipes use 1 yeast but some have 2+
                yeastName.append(yeast_container.find_all('thead')[i].find('span').text.strip())
                #table is in a weird format compared to the others, also varies in dimensions
                #so we'll take in the entire text of the table and parse it
                unparsed = yeast_container.find_all('tbody')[i].find_all('tr')[0].find_all('td')[0].text
                #replace any whitespace with just 1 space
                unparsed = ' '.join(unparsed.split())
                #make it comma delimited so we can make a list of features
                unparsed = unparsed.replace('Attenuation', ', Attenuation').replace('Flocculation', ', Flocculation')
                unparsed = unparsed.replace('Optimum', ', Optimum').replace('Starter', ', Starter')
                unparsed = unparsed.replace('Fermentation', ', Fermentation').replace('Pitch', ', Pitch')
                unparsed = unparsed.split(',')

                #depending on what feature each element of the list is, assign respective value
                for i in range(0,len(unparsed)):
                    if 'Amount:' in unparsed[i]:
                        yeastAmount.append(unparsed[i].split('Amount:')[1].strip())
                    elif 'Attenuation (avg):' in unparsed[i]:
                        yeastAttenuation.append(unparsed[i].split('Attenuation (avg):')[1].strip()+' (average)')
                    elif 'Attenuation (custom):' in unparsed[i]:
                        yeastAttenuation.append(unparsed[i].split('Attenuation (custom):')[1].strip()+' (custom)')
                    elif 'Flocculation:' in unparsed[i]:
                        yeastFloc.append(unparsed[i].split('Flocculation:')[1].strip())
                    elif 'Optimum Temp:' in unparsed[i]:
                        yeastOptTemp.append(unparsed[i].split('Optimum Temp:')[1].strip())
                    elif 'Starter:' in unparsed[i]:
                        yeastStarter.append(unparsed[i].split('Starter:')[1].strip())
                    elif 'Fermentation Temp:' in unparsed[i]:
                        yeastFermTemp.append(unparsed[i].split('Fermentation Temp:')[1].strip())
                    elif 'Pitch Rate:' in unparsed[i]:
                        yeastPR.append(unparsed[i].split('Pitch Rate:')[1].strip())
                    else:
                        print("Theres a yeast feature you're unaware of: ", link)

                #insert 'NA' if features aren't available
                if not any('Amount' in x for x in unparsed):
                    yeastAmount.append('NA')
                elif not any('Attenuation' in x for x in unparsed):
                    yeastAttenuation.append('NA')
                elif not any('Flocculation' in x for x in unparsed):
                    yeastFloc.append('NA')
                elif not any('Optimum Temp' in x for x in unparsed):
                    yeastOptTemp.append('NA')
                elif not any('Starter' in x for x in unparsed):
                    yeastStarter.append('NA')
                elif not any('Fermentation Temp' in x for x in unparsed):
                    yeastFermTemp.append('NA')
                elif not any('Pitch Rate' in x for x in unparsed):
                    yeastPR.append('NA')
        
        yeastNames.append(yeastName)
        yeastAmounts.append(yeastAmount)
        yeastAttenuations.append(yeastAttenuation)
        yeastFlocs.append(yeastFloc)
        yeastOptTemps.append(yeastOptTemp)
        yeastStarters.append(yeastStarter)
        yeastFermTemps.append(yeastFermTemp)
        yeastPRs.append(yeastPR)

        #get priming info
        priming_container = soup.find('div', class_='brewpart',  id = "primingmethod")
        if priming_container is None:
            priming.append('NA')
        else:
            priming.append(priming_container.find('td').text.strip().replace('\xa0', '').replace('\t','').replace('\n', ','))

        #get other ingredients
        otherAmount = []
        otherName = []
        otherType = []
        otherUse = []
        otherTime = []

        other_container = soup.find('div', class_ = "brewpart", id = 'others')
        if other_container is None:
            otherAmount.append('NA')
            otherName.append('NA')
            otherType.append('NA')
            otherUse.append('NA')
            otherTime.append('NA')
        else:
            for i in range(1,len(other_container.table.find_all('tr'))):  # i = table row, j = table column 
                for j in range(0,len(other_container.table.find_all('tr')[i])):
                    if j == 0: #amounts are in 1st column
                        otherAmount.append(other_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 1: #names in 2nd column...etc
                        #similar problem here as with hopNames and fermNames, see them for more details
                        if other_container.table.find_all('tr')[i].find_all('td')[j].find('a') is None:
                            otherName.append(other_container.table.find_all('tr')[i].find_all('td')[j].find('span').text.strip())
                        else:
                            otherName.append(other_container.table.find_all('tr')[i].find_all('td')[j].find('a').text.strip())
                    elif j == 2: #type
                        otherType.append(other_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 3: #use
                        otherUse.append(other_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 4: #time
                        otherTime.append(other_container.table.find_all('tr')[i].find_all('td')[j].text.strip())

        otherNames.append(otherName)
        otherAmounts.append(otherAmount)
        otherTypes.append(otherType)
        otherUses.append(otherUse)
        otherTimes.append(otherTime)

        #get target water profile
        water_container = soup.find('div',class_='brewpart',id='water')
        if water_container is None:
            ca.append('NA')
            mg.append('NA')
            na.append('NA')
            cl.append('NA')
            so4.append('NA')
            hco3.append("NA")
        else:
            ca.append(water_container.find('table').find_all('td')[0].text.strip())
            mg.append(water_container.find('table').find_all('td')[1].text.strip())
            na.append(water_container.find('table').find_all('td')[2].text.strip())
            cl.append(water_container.find('table').find_all('td')[3].text.strip())
            so4.append(water_container.find('table').find_all('td')[4].text.strip())
            hco3.append(water_container.find('table').find_all('td')[5].text.strip())


        #get notes 
        notes_container = soup.find('div',class_='ui message')
        if notes_container is None:
            notes.append('NA')
        else:
            notes.append(notes_container.text.strip())

        #get number of views recipe has and number of times other users have brewed recipe
        viewsAndBrews_container = soup.find('div', class_='ui two statistics')
        if viewsAndBrews_container is None:
            views.append('NA')
            brews.append('NA')
        else:
            views.append(viewsAndBrews_container.find_all('div',class_='value')[0].text.strip())
            brews.append(viewsAndBrews_container.find_all('div',class_='value')[1].text.strip())

        #save file every 250 recipes
        if (count % 50) == 0:
            df = pd.DataFrame({'recipeURL':recipeURL, "name":name, "author":author, 'dateCreated':dateCreated,
                               'urlProvided':links,'views':views,'brews':brews,'OG':og, 'FG':fg, 'ABV':abv,
                               'IBU':ibu,'SRM':srm,'mashpH':mashPh,"method":method, 'style':style,
                               'boilTime':boilTime, 'batchSize': batchSize,'preBoilSize':preBoilSize, 
                               'postBoilSize': postBoilSize, 'preBoilGravity':preBoilGravity, 'efficency':efficiency,
                               'source':source,'rating':rating,'ratingCount':numRating, 'noChill':noChill, 
                               'hopUtilization':hopUtil,'calories':calories, 'carbs':carbs,'fermNames':fermNames, 
                               'fermAmounts': fermAmounts,'fermPPGs': fermPPGs,'fermDegreeLovibond': fermLs, 
                               'fermPercentages':fermBillPs,'hopNames': hopNames, 'hopAmounts': hopAmounts, 
                               'hopForms': hopForms, 'hopAAs': hopAAs,'hopUses':hopUses, 'hopTimes': hopTimes,
                               'hopIBUs': hopIBUs, 'hopPercentages':hopBillPs,'otherIngredients':otherNames, 
                               'otherAmounts':otherAmounts,'otherTypes':otherTypes,'otherUses':otherUses,
                               'otherTimes':otherTimes, 'Ca':ca,'Mg':mg,'Na':na,'Cl':cl,'SO4':so4,'HCO3':hco3,
                               'primingInfo':priming,'yeastNames':yeastNames, 'yeastAmounts':yeastAmounts,
                               'yeastAttenuations': yeastAttenuations, 'yeastOptTemps':yeastOptTemps,
                               'yeastFlocculations':yeastFlocs,'yeastStarters':yeastStarters,
                               'yeastFermTemps':yeastFermTemps,'yeastPitchRates':yeastPRs,'Notes':notes})
        
            df.to_csv('NEIPA'+str(count)+'.csv', encoding = 'utf-8', index = False)
            
        
        # pause loop to avoid overloading server
        #time.sleep(random.randint(5,9))
        time.sleep(random.randint(3,6))

print("--- %s seconds ---" % (time.time() - start_time))
#takes 13 seconds to scrape 5 pages <==> 2.6s to scrape each page
    


--- 4663.68409204483 seconds ---


In [182]:
data = pd.read_csv('NEIPAfinal 2-20-2020.csv')
data.head()

,recipeURL,name,author,dateCreated,urlProvided,views,brews,OG,FG,ABV,...,primingInfo,yeastNames,yeastAmounts,yeastAttenuations,yeastOptTemps,yeastFlocculations,yeastStarters,yeastFermTemps,yeastPitchRates,notes
0,https://www.brewersfriend.com/homebrew/recipe/...,Avg. Perfect Northeast IPA (NEIPA),kcq101,Tuesday May 31st 2016,NaN,"243,420",296.0,1.062,1.013,6.50%,...,"Method: Keg - Force Carb.,CO2 Level: 2.4 Volumes",['Wyeast - London Ale III 1318'],['1'],"['78% (custom)', 'NA']",['64 - 74 °F'],['High'],['No'],[],[],Fermentation Dry Hop @ 70% attenuation (1.027)...
1,https://www.brewersfriend.com/homebrew/recipe/...,Bonnie and Clyde Mango Milkshake NEIPA. We use...,Vallka,Friday September 15th 2017,NaN,"5,462",7.0,1.063,1.02,5.65%,...,CO2 Level: 2.35 Volumes,['Wyeast - London Ale III 1318'],['1'],"['73% (custom)', 'NA']",['18 - 23 °C'],['High'],['No'],['20 °C'],['0.35 (M cells / ml / ° P) 124 B cells requir...,John's ale yeast
2,https://www.brewersfriend.com/homebrew/recipe/...,#SerNEpper,Mazsi,Thursday October 26th 2017,NaN,"4,268",0.0,1.062,1.014,6.30%,...,NaN,['The Yeast Bay - Vermont Ale'],['1'],['78.5% (average)'],['18 - 22 °C'],['Med/Low'],['No'],['21 °C'],['0.75 (M cells / ml / ° P) 422 B cells requir...,élesztő:\nWyeast - London Ale III\nWLP Burling...
3,https://www.brewersfriend.com/homebrew/recipe/...,Anniversary N. E. IPA,Ostrander,Saturday September 29th 2012,NaN,"3,895",1.0,1.066,1.018,6.26%,...,"Method: Corn Sugar,Amount: 1.4 oz",['Wyeast - London Ale III 1318'],['1'],['73% (average)'],['64 - 74 °F'],['High'],['No'],['64 °F'],['0.75 (M cells / ml / ° P) 298 B cells requir...,Invert sugar syrup added to primary fermentor ...
4,https://www.brewersfriend.com/homebrew/recipe/...,DIPA v.1,Beatsontoast,Monday September 30th 2013,NaN,"3,795",0.0,1.068,1.015,7.33%,...,CO2 Level: 2.25 Volumes,['Fermentis - Safale - American Ale Yeast US-0...,"['1', '1']","['81% (average)', '76.5% (average)']","['12 - 25 °C', '20 - 23 °C']","['Medium', 'Medium']","['No', 'No']","['20 °C', '20 °C']",['0.5 (M cells / ml / ° P) 199 B cells require...,NaN


# Fermentables Exploratory Analysis

In [183]:
#get list of column names for reference
print(data.columns)

Index(['recipeURL', 'name', 'author', 'dateCreated', 'urlProvided', 'views',
       'brews', 'OG', 'FG', 'ABV', 'IBU', 'SRM', 'mashpH', 'method', 'style',
       'boilTime', 'batchSize', 'preBoilSize', 'postBoilSize',
       'preBoilGravity', 'efficency', 'source', 'rating', 'ratingCount',
       'noChill', 'hopUtilization', 'calories', 'carbs', 'fermNames',
       'fermAmounts', 'fermPPGs', 'fermDegreeLovibond', 'fermPercentages',
       'hopNames', 'hopAmounts', 'hopForms', 'hopAAs', 'hopUses', 'hopTimes',
       'hopIBUs', 'hopPercentages', 'otherIngredients', 'otherAmounts',
       'otherTypes', 'otherUses', 'otherTimes', 'Ca', 'Mg', 'Na', 'Cl', 'SO4',
       'HCO3', 'primingInfo', 'yeastNames', 'yeastAmounts',
       'yeastAttenuations', 'yeastOptTemps', 'yeastFlocculations',
       'yeastStarters', 'yeastFermTemps', 'yeastPitchRates', 'notes'],
      dtype='object')


In [224]:
fermList[0:10][0]

['American - Pale 2-Row',
 'American - Wheat',
 'Flaked Oats',
 'Canadian - Honey Malt']

In [227]:
#get list of fermentables used in all the recipes (contains repeats)
fermList = []

#for each recipe, turn string of all ingredients into a list and add it to list of all fermentables used
#this method assumes that each ingredient is only present once in each recipe
for i in range(0,len(data.fermNames)):
    test_string = data.fermNames[i]
    tempType = re.findall(r'\'(.*?)\'', test_string)
    resType = list(map(str, tempType))    
    fermList.append(resType)

#create comprehensive list of unique fermentables used
#bill refers to the list of all fermentables in a recipe
uniqueFerm = []
for bill in fermList:
    for ferm in bill:    
        if ferm not in uniqueFerm:
            uniqueFerm.append(ferm)

print('Number of unique fermentables: ',len(uniqueFerm))

#find number of recipes that each unique fermentable is used in.
#initialize counts for each unique fermentable to zero
uniqueFermCount = [0]*len(uniqueFerm)

for i in range(0,len(fermList)):
    for j in range(0,len(set(fermList[i]))):
        if fermList[i][j] in uniqueFerm:
            uniqueFermCount[uniqueFerm.index(fermList[i][j])] = uniqueFermCount[uniqueFerm.index(fermList[i][j])]+1

#find percentage of recipes that contain the unique fermentable
fermPercentage = [x / len(data.fermNames) for x in uniqueFermCount]

ferm_df = pd.DataFrame({'Fermentable':uniqueFerm,'Count':uniqueFermCount, 'Percentage':fermPercentage})
ferm_df = ferm_df.sort_values('Count',ascending = False).reset_index(drop=True)

ferm_df[0:10]

Number of unique fermentables:  474


,Fermentable,Count,Percentage
0,Flaked Oats,1890,0.662925
1,American - Pale 2-Row,1227,0.430375
2,Flaked Wheat,829,0.290775
3,American - White Wheat,496,0.173974
4,United Kingdom - Maris Otter Pale,377,0.132234
5,Canadian - Honey Malt,364,0.127675
6,American - Carapils (Dextrine Malt),362,0.126973
7,United Kingdom - Golden Promise,319,0.111891
8,American - Pilsner,272,0.095405
9,American - Wheat,270,0.094704


In [273]:
def getIngredientCounts(ingrSeries, ingrName):
    #get list of ingredients used (contains repeats)
    ingrList = []

    #for each recipe, turn string of all ingredients into a list and add it to list of all fermentables used
    #this method assumes that each ingredient is only present once in each recipe
    for i in range(0,len(ingrSeries)):
        test_string = ingrSeries[i]
        tempType = re.findall(r'\'(.*?)\'', test_string)
        resType = list(map(str, tempType))    
        ingrList.append(resType)

    #create comprehensive list of unique hops used
    ######################################## USE SET instead??? ###########
    uniqueIngr = []
    for ingrs in ingrList:
        for ingr in ingrs:    
            if ingr not in uniqueIngr:
                uniqueIngr.append(ingr)

    print('Number of unique',ingrName,': ', len(uniqueIngr)) 

    #find number of recipes that each unique grain is used in.
    uniqueIngrCount = [0]*len(uniqueIngr)

    for i in range(0,len(ingrList)):
        for j in range(0,len(set(ingrList[i]))): #use set() to get set of hops for each recipe i.e set([1,1,2,2,3]) = [1,2,3]
            if ingrList[i][j] in uniqueIngr:
                uniqueIngrCount[uniqueIngr.index(ingrList[i][j])] = uniqueIngrCount[uniqueIngr.index(ingrList[i][j])]+1

    ingrPercentage = [x / len(ingrSeries) for x in uniqueIngrCount]            

    ingr_df = pd.DataFrame({ingrName:uniqueIngr,'Count':uniqueIngrCount, 'Percentage':ingrPercentage})
    ingr_df = ingr_df.sort_values('Count',ascending = False).reset_index(drop=True)
    
    return ingr_df

In [263]:
def plot_histogram(some_list):
    n, bins, patches = plt.hist(x=some_list, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
    plt.grid(axis='y', alpha=0.75)
    plt.xlabel(some_list.name)
    plt.ylabel('frequency')

    print("mean: ", np.nanmean(some_list))
    print("median: ", np.nanmedian(some_list))
    print("Std dev: ", np.sqrt(np.nanvar(some_list)))

In [274]:
# plt.subplot(3,3,1)
# # plot_histogram(data.OG)
# # plot_histogram(data.FG)
# plot_histogram(data.ABV)
# plot_histogram(data.IBU)
# plot_histogram(data.mashpH)
# plot_histogram(data.SRM)
# plot_histogram(data.views)
# plot_histogram(data.brews)
# plt.tight_layout()


In [275]:
hopCount = getIngredientCounts(data.hopNames, 'Hops')
fermCount = getIngredientCounts(data.fermNames, 'Fermentables')
otherCount = getIngredientCounts(data.otherIngredients, 'Others')
dfCount = pd.concat([hopCount,fermCount,otherCount],axis=1)
dfCount[0:10]

Number of unique Hops :  603
Number of unique Fermentables :  474
Number of unique Others :  310


,Hops,Count,Percentage,Fermentables,Count,Percentage,Others,Count,Percentage
0,Citra,1586,0.556296,Flaked Oats,1887.0,0.661873,NA,1799.0,0.631007
1,Mosaic,1106,0.387934,American - Pale 2-Row,1224.0,0.429323,Gypsum,652.0,0.228692
2,Amarillo,596,0.209049,Flaked Wheat,826.0,0.289723,Calcium Chloride,338.0,0.118555
3,Galaxy,529,0.185549,American - White Wheat,496.0,0.173974,Calcium Chloride (dihydrate),225.0,0.078920
4,Simcoe,426,0.149421,United Kingdom - Maris Otter Pale,374.0,0.131182,Lactic acid,217.0,0.076114
5,El Dorado,410,0.143809,Canadian - Honey Malt,363.0,0.127324,Epsom Salt,205.0,0.071905
6,Centennial,279,0.097860,American - Carapils (Dextrine Malt),362.0,0.126973,Table Salt,157.0,0.055068
7,Cascade,275,0.096457,United Kingdom - Golden Promise,319.0,0.111891,Wyeast - Beer Nutrient,97.0,0.034023
8,Columbus,263,0.092248,American - Pilsner,271.0,0.095054,Whirlfloc,89.0,0.031217
9,NA,250,0.087689,American - Wheat,270.0,0.094704,Phosphoric acid,84.0,0.029463


# Hop Exploratory Analysis

In [222]:
#get list of fermentables used (contains repeats)
hopList = []

#for each recipe, turn string of all ingredients into a list and add it to list of all fermentables used
#this method assumes that each ingredient is only present once in each recipe
for i in range(0,len(data.hopNames)):
    test_string = data.hopNames[i]
    tempType = re.findall(r'\'(.*?)\'', test_string)
    resType = list(map(str, tempType))    
    hopList.append(resType)

#create comprehensive list of unique hops used
######################################## USE SET instead??? ###########
uniqueHop = []
for hops in hopList:
    for hop in hops:    
        if hop not in uniqueHop:
            uniqueHop.append(hop)

print('Number of unique hops: ', len(uniqueHop)) # 113

#find number of recipes that each unique grain is used in.
uniqueHopCount = [0]*len(uniqueHop)

for i in range(0,len(hopList)):
    for j in range(0,len(set(hopList[i]))): #use set() to get set of hops for each recipe i.e set([1,1,2,2,3]) = [1,2,3]
        if hopList[i][j] in uniqueHop:
            uniqueHopCount[uniqueHop.index(hopList[i][j])] = uniqueHopCount[uniqueHop.index(hopList[i][j])]+1

hopPercentage = [x / len(data.hopNames) for x in uniqueHopCount]            

hop_df = pd.DataFrame({'Hop':uniqueHop,'Count':uniqueHopCount, 'Percentage':hopPercentage})
hop_df = hop_df.sort_values('Count',ascending = False).reset_index(drop=True)
hop_df[0:10]

Number of unique hops:  603


,Hop,Count,Percentage
0,Citra,1586,0.556296
1,Mosaic,1106,0.387934
2,Amarillo,596,0.209049
3,Galaxy,529,0.185549
4,Simcoe,426,0.149421
5,El Dorado,410,0.143809
6,Centennial,279,0.097860
7,Cascade,275,0.096457
8,Columbus,263,0.092248
9,NA,250,0.087689


In [ ]:
#Standardize hop weights to oz
standardizedWeights = []
for i in range(0, len(data.hopAmounts)):
    wgts = []
    test_string = data.hopAmounts[i]
    tempType = re.findall(r'\'(.*?)\'', test_string)
    resType = list(map(str, tempType))
    for j in range(0, len(resType)):
        if 'kg' in resType[j]:
            wgt = resType[j].split(' kg')[0]
            wgt = float(wgt)*35.274 #convert to oz, 35.274 oz per kg
        elif 'g' in resType[j]:
            wgt = resType[j].split(' g')[0]
            wgt = float(wgt)*0.035274 #convert to oz, 0.0352 oz per g
        elif 'lb' in resType[j]:
            wgt = resType[j].split(' lb')[0]
            wgt = float(wgt)*16 #convert to oz, 16 oz per lb
        elif 'oz' in resType[j]:
            wgt = resType[j].split(' oz')[0]
            wgt = float(wgt)
        else:
            wgt = 'NA'
        
        wgts.append(wgt)

    standardizedWeights.append(wgts)
print('Checking conversions to ounces:')    
print(data.hopAmounts[38:41])
print(standardizedWeights[38:41])